In [1]:
import sys
import pandas as pd
import sqlparse
import sqlite3

In [2]:
%load_ext jupyter_black

In [3]:
# import nltk
# nltk.download('punkt')

In [4]:
sys.path.append("/home/ec2-user/SageMaker/spider")

In [5]:
sys.path

['/home/ec2-user/SageMaker/merck-text2sql-genai-poc/notebooks',
 '/home/ec2-user/SageMaker/.cs/conda/envs/br_py310/lib/python310.zip',
 '/home/ec2-user/SageMaker/.cs/conda/envs/br_py310/lib/python3.10',
 '/home/ec2-user/SageMaker/.cs/conda/envs/br_py310/lib/python3.10/lib-dynload',
 '',
 '/home/ec2-user/SageMaker/.cs/conda/envs/br_py310/lib/python3.10/site-packages',
 '/home/ec2-user/SageMaker/spider']

In [6]:
from evaluation import Evaluator
from process_sql import tokenize, get_schema, get_sql, Schema

In [7]:
db = "../database/synpuf.db"

In [9]:
df_sample = pd.read_excel("../data/synpuf_query_new.xlsx", sheet_name="Sheet1")

In [10]:
queries_gt = df_sample["Synpuf query"].tolist()

In [11]:
queries_gt_mod = [q.replace("rwdex_raw_synpuf.", "") for q in queries_gt]

In [12]:
queries_gt_mod

['select count(distinct desynpuf_id) from outpatient_claims;',
 'select count(distinct desynpuf_id) from inpatient_claims;',
 "select distinct desynpuf_id from inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'",
 "select distinct desynpuf_id from outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'",
 "select distinct desynpuf_id from outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01' ",
 'select distinct desynpuf_id from inpatient_claims where admtng_icd9_dgns_cd in (select icd9_cd from diabetes_code_list)',
 'select desynpuf_id where datediff(year, bene_birth_dt, bene_death_dt) >= 30 or bene_death_dt is null',
 'select desynpuf_id where datediff(year, bene_birth_dt, bene_death_dt) <= 30',
 'select distinct desynpuf_id from outpatient_claims where \nicd9_dgns_cd_1 in (select icd9_cd from diabetes_code_l

In [29]:
evaluator = Evaluator()

In [13]:
def sql_pd(q, db_name):
    with sqlite3.connect(db_name) as conn:
        df_res = pd.read_sql(q, conn)
    return df_res

In [14]:
schema = Schema(get_schema(db))

In [15]:
schema.schema["inpatient_claims"]

['desynpuf_id',
 'clm_id',
 'segment',
 'clm_from_dt',
 'clm_thru_dt',
 'prvdr_num',
 'clm_pmt_amt',
 'nch_prmry_pyr_clm_pd_amt',
 'at_physn_npi',
 'op_physn_npi',
 'ot_physn_npi',
 'clm_admsn_dt',
 'admtng_icd9_dgns_cd',
 'clm_pass_thru_per_diem_amt',
 'nch_bene_ip_ddctbl_amt',
 'nch_bene_pta_coinsrnc_lblty_am',
 'nch_bene_blood_ddctbl_lblty_am',
 'clm_utlztn_day_cnt',
 'nch_bene_dschrg_dt',
 'clm_drg_cd',
 'icd9_dgns_cd_1',
 'icd9_dgns_cd_2',
 'icd9_dgns_cd_3',
 'icd9_dgns_cd_4',
 'icd9_dgns_cd_5',
 'icd9_dgns_cd_6',
 'icd9_dgns_cd_7',
 'icd9_dgns_cd_8',
 'icd9_dgns_cd_9',
 'icd9_dgns_cd_10',
 'icd9_prcdr_cd_1',
 'icd9_prcdr_cd_2',
 'icd9_prcdr_cd_3',
 'icd9_prcdr_cd_4',
 'icd9_prcdr_cd_5',
 'icd9_prcdr_cd_6',
 'hcpcs_cd_1',
 'hcpcs_cd_2',
 'hcpcs_cd_3',
 'hcpcs_cd_4',
 'hcpcs_cd_5',
 'hcpcs_cd_6',
 'hcpcs_cd_7',
 'hcpcs_cd_8',
 'hcpcs_cd_9',
 'hcpcs_cd_10',
 'hcpcs_cd_11',
 'hcpcs_cd_12',
 'hcpcs_cd_13',
 'hcpcs_cd_14',
 'hcpcs_cd_15',
 'hcpcs_cd_16',
 'hcpcs_cd_17',
 'hcpcs_cd_18',

In [16]:
print(sqlparse.format(queries_gt_mod[0], reindent=True, keyword_case="upper"))

SELECT count(DISTINCT desynpuf_id)
FROM outpatient_claims;


In [19]:
sql_pprint = lambda q: print(sqlparse.format(q, reindent=True, keyword_case="upper"))

In [31]:
get_sql(schema, queries_gt_mod[0])

{'from': {'table_units': [('table_unit', '__outpatient_claims__')],
  'conds': []},
 'select': (False,
  [(3, (0, (0, '__outpatient_claims.desynpuf_id__', True), None))]),
 'where': [],
 'groupBy': [],
 'having': [],
 'orderBy': [],
 'limit': None,
 'intersect': None,
 'union': None,
 'except': None}

In [33]:
evaluator.eval_hardness(get_sql(schema, queries_gt_mod[0]))

'easy'

In [36]:
for i, q in enumerate(queries_gt_mod):
    try:
        hardness = evaluator.eval_hardness(get_sql(schema, q))
        print(f"t{i+1}: {hardness}")
    except Exception as e:
        print(f"t{i+1}", e)

t1: easy
t2: easy
t3: easy
t4: easy
t5: hard
t6 'diabetes_code_list'
t7 'from' not found
t8 'from' not found
t9 'diabetes_code_list'
t10 'a'
t11 'a'
t12 'a'
t13 'a'
t14 Error col: as
t15 Error col: as
t16 Error col: as
t17 Error col: as
t18 'prcdr_code_list'
t19 'a'
t20 'a'
t21 'a'
t22 'a'
t23: easy
t24 'inner'
t25 'a'
t26 'a'
t27 'a'
t28 'a'
t29 'a'
t30 'a'
t31 'a'
t32 'a'
t33 Error col: as
t34 'a'
t35 'a'
t36 Error col: admtng_icd10_dgns_cd
t37 Error col: datediff
t38 Error col: datediff
t39 Error col: datediff
t40 'a'


In [23]:
for i, q in enumerate(queries_gt_mod):
    try:
        # print(f"t{i+1}", sql_pd(q, db))
        sql_pd(q, db)
    except Exception as e:
        # print(f"t{i+1}", sql_pprint(q), e)
        print(sql_pprint(q))

SELECT DISTINCT desynpuf_id
FROM inpatient_claims
WHERE admtng_icd9_dgns_cd IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
None
SELECT desynpuf_id
WHERE datediff(YEAR, bene_birth_dt, bene_death_dt) >= 30
  OR bene_death_dt IS NULL
None
SELECT desynpuf_id
WHERE datediff(YEAR, bene_birth_dt, bene_death_dt) <= 30
None
SELECT DISTINCT desynpuf_id
FROM outpatient_claims
WHERE icd9_dgns_cd_1 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_2 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_3 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_4 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_5 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_6 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_7 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_8 IN
    (SELECT icd9_cd
     FROM diabetes_code_list)
  OR icd9_dgns_cd_9 IN
    (SELECT icd9_c

In [45]:
queries_gt_mod[21]

'select distinct a.desynpuf_id, a.min(clm_admsn_dt) dt1, b.min(clm_admsn_dt) dt2 from inpatient_claims a inner join inpatient_claims b on a.desynpuf_id = b.desynpuf_id and a.admtng_icd10_dgns_cd in code_list_1 and b.admtng_icd10_dgns_cd in code_list_2 and dt1 < dt2'

In [46]:
print(sqlparse.format(queries_gt_mod[21], reindent=True, keyword_case="upper"))

SELECT DISTINCT a.desynpuf_id,
                a.min(clm_admsn_dt) dt1,
                b.min(clm_admsn_dt) dt2
FROM inpatient_claims a
INNER JOIN inpatient_claims b ON a.desynpuf_id = b.desynpuf_id
AND a.admtng_icd10_dgns_cd IN code_list_1
AND b.admtng_icd10_dgns_cd IN code_list_2
AND dt1 < dt2


In [51]:
print(sqlparse.format(queries_gt_mod[36], reindent=True, keyword_case="upper"))

SELECT desynpuf_id,
       min(clm_from_dt),
       max(clm_through_dt),
       datediff(DAY, max(clm_through_dt), clm_from_dt)
FROM inpatient_claims
GROUP BY desynpuf_id


In [27]:
q = """
SELECT a.desynpuf_id,
       min(a.srvc_dt)
FROM prescription_drug_events a, inpatient_claims b
WHERE a.desynpuf_id = b.desynpuf_id  AND a.prod_srvc_id = 'xxx'GROUP BY a.desynpuf_id
"""
sql_pd(q, db)

Empty DataFrame
Columns: [DESYNPUF_ID, min(a.srvc_dt)]
Index: []